# 0. import packages

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
import csv
import import_ipynb
import Lib3

importing Jupyter notebook from Lib3.ipynb


In [124]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/elliedam/.wdm/drivers/chromedriver/mac64/99.0.4844.51]
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Foody_TPHCM

# 1. Get links and names of restaurant

In [ ]:
driver.get('https://www.foody.vn')

In [ ]:
# lấy tên và link của các nhà hàng
for i in range (1,50):
    try:
        driver.find_elements_by_class_name('fd-btn-more')[0].click()
        sleep(2)
    except IndexError:
        pass
    continue
res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")
res_names = [x.text for x in res]
links = [x.get_attribute('href') for x in res]

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2824142087.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2824142087.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")


In [ ]:
print('Số lượng nhà hàng',len(res_names))
res_names[:5]

Số lượng nhà hàng 600


['Chícc Chícc - Gà Rán Hàn Quốc',
 'Thành Mập - Chân Gà Rút Xương Ngâm Sả Tắc - Bạch Đằng - Shop Online',
 'TocoToco Bubble Tea - Phạm Hùng',
 '2 Béo - Bánh Sầu Riêng - Shop Online',
 'Bánh 9 Sạch - Bánh Sầu Riêng - Shop Online']

In [ ]:
# add thêm '/binh-luan' ở cuối link để truy cập vào phần bình luận của mỗi nhà hàng
links_bl = []
for l in links:
    links_bl.append(l +'/binh-luan')
print('Số lượng link:',len(links_bl))
links_bl[:5]

Số lượng link: 600


['https://www.foody.vn/ho-chi-minh/chicc-chicc-ga-ran-han-quoc/binh-luan',
 'https://www.foody.vn/ho-chi-minh/thanh-map-chan-ga-rut-xuong-ngam-sa-tac-bach-dang-shop-online/binh-luan',
 'https://www.foody.vn/ho-chi-minh/tocotoco-bubble-tea-pham-hung/binh-luan',
 'https://www.foody.vn/ho-chi-minh/2-beo-banh-sau-rieng-shop-online/binh-luan',
 'https://www.foody.vn/ho-chi-minh/banh-9-sach-banh-sau-rieng-shop-online/binh-luan']

In [ ]:
# Save results to csv
with open('data/foody_links_bl.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(links_bl)

with open('data/foody_res_names.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(res_names)

# 2. Review & Score scraping

In [ ]:
# load links
links_bl_load = pd.read_csv('data/foody_links_bl.csv', header=None)
links_bl_load = list(links_bl_load.iloc[0,:].values)
links_bl_load[:5]

['https://www.foody.vn/ho-chi-minh/chicc-chicc-ga-ran-han-quoc/binh-luan',
 'https://www.foody.vn/ho-chi-minh/thanh-map-chan-ga-rut-xuong-ngam-sa-tac-bach-dang-shop-online/binh-luan',
 'https://www.foody.vn/ho-chi-minh/tocotoco-bubble-tea-pham-hung/binh-luan',
 'https://www.foody.vn/ho-chi-minh/2-beo-banh-sau-rieng-shop-online/binh-luan',
 'https://www.foody.vn/ho-chi-minh/banh-9-sach-banh-sau-rieng-shop-online/binh-luan']

In [ ]:
# load restaurant names
res_names_load = pd.read_csv('data/foody_res_names.csv', header=None)
res_names_load = list(res_names_load.iloc[0,:].values)
res_names_load[:5]

['Chícc Chícc - Gà Rán Hàn Quốc',
 'Thành Mập - Chân Gà Rút Xương Ngâm Sả Tắc - Bạch Đằng - Shop Online',
 'TocoToco Bubble Tea - Phạm Hùng',
 '2 Béo - Bánh Sầu Riêng - Shop Online',
 'Bánh 9 Sạch - Bánh Sầu Riêng - Shop Online']

In [ ]:
len(links_bl_load), len(res_names_load)

(600, 600)

Vì số lượng nhà hàng lớn, ta sẽ chia làm 6 lần, mỗi lần 100 nhà hàng

## Data scraping 1

In [ ]:
# data scraping 100 nhà hàng đầu tiên [0:100]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(0,100):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2853847012.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2853847012.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span")
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2853847012.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span")


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

3452 3452 3452


In [ ]:
# Create dataframe and save to csv file
dict_foody1 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df1 = pd.DataFrame(dict_foody1)

In [ ]:
df1.head()

,name,review_text,review_score
0,Chícc Chícc - Gà Rán Hàn Quốc,"Gà tắm mắm, phô mai kéo sợi siêu ngon, giá mềm",10
1,Chícc Chícc - Gà Rán Hàn Quốc,"Gà BBQ Hàn, phô mai kéo sợi siêu ngon, giá mềm",10
2,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán phô mai kéo sợi sốt BBQ, siu ngon",10
3,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán kem bơ tỏi, ngon, lạ, béo, giá ổn áp",10
4,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán kem bơ tỏi, ngon, lạ, béo, giá ổn áp",10


In [ ]:
df1.shape

(3452, 3)

In [ ]:
df1.to_csv('data/foodyData1.csv')

## Data scraping 2

In [ ]:
# data scraping 100 nhà hàng tiếp theo [100:200]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(100,200):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1652 1652 1652


In [ ]:
# Create dataframe and save to csv file
dict_foody2 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df2 = pd.DataFrame(dict_foody2)

In [ ]:
df2.head()

,name,review_text,review_score
0,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Nhớ hương vị gần chục năm trước ăn nên nay ord...,7.2
1,Trà Sữa Feeling Tea - Đường 3 Tháng 2,"Mình uống trà sữa nhìu chỗ rồi, nhưng uống hợp...",7.6
2,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Chạy đi làm về ngang quài nghe mùi trà thấy th...,9.0
3,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Order món hồng trà vải tươi và trà xanh vải tư...,6.0
4,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Rõ ràng mình order và trả tiền cho 3 ly trà sữ...,1.0


In [ ]:
df2.shape

(1652, 3)

In [ ]:
df2.to_csv('data/foodyData2.csv')

## Data scraping 3

In [ ]:
# data scraping 100 nhà hàng tiếp theo [200:300]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(200,300):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

2046 2046 2046


In [ ]:
# Create dataframe and save to csv file
dict_foody3 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df3 = pd.DataFrame(dict_foody3)

In [ ]:
df3.head()

,name,review_text,review_score
0,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Phục vụ quá lâu,2.6
1,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Nhìn khá giống bánh hỏi thịt nướng phải không ...,9.8
2,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,hôm nay có mua về 2 hôp com chai giá 62k một h...,3.8
3,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Đây là một trong những quán chay mình rất thíc...,9.6
4,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Lần đầu đi ăn mà cảm thấy thật sự hài lòng! Mì...,6.2


In [ ]:
df3.shape

(2046, 3)

In [ ]:
df3.to_csv('data/foodyData3.csv')

## Data scraping 4

In [ ]:
# data scraping 100 nhà hàng tiếp theo [300:400]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(300,400):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

2732 2732 2732


In [ ]:
# Create dataframe and save to csv file
dict_foody4 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df4 = pd.DataFrame(dict_foody4)

In [ ]:
df4.head()

,name,review_text,review_score
0,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,NgonnnnnnnnnnNNgonnnnnnnnnngonnnnnnnnnn,10
1,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Ngonnnnnnnnnn,10
2,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Ngonnnnnnnnnn,10
3,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Ngonnnnnnnnnn,10
4,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Ngonnnnnnnnnn,10


In [ ]:
df4.shape

(2732, 3)

In [ ]:
df4.to_csv('data/foodyData4.csv')

## Data scraping 5

In [ ]:
# data scraping 100 nhà hàng tiếp theo [400:500]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(400,500):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1181 1181 1181


In [ ]:
# Create dataframe and save to csv file
dict_foody5 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df5 = pd.DataFrame(dict_foody5)
df5.head()

,name,review_text,review_score
0,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,"Mua cho mấy anh em trong công ty ăn thử, ai cũ...",9.4
1,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,"Đã ăn nhiều nơi, nhưng bánh bao của shop là hợ...",10
2,Chè Ngọc - Đường D9,"Nước cốt dừa chè bưởi bị chua hư,chè thốt nốt ...",3.0
3,Chè Ngọc - Đường D9,"Quán làm món khá ngon và chất lượng, đã đặt kh...",6.6
4,Chè Ngọc - Đường D9,"Giá cả hợp lí, chè ngon, sẽ ủng hộ quán thường...",6.0


In [ ]:
df5.shape

(1181, 3)

In [ ]:
df5.to_csv('data/foodyData5.csv')

## Data scraping 6

In [ ]:
# data scraping 100 nhà hàng tiếp theo [500:600]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(500,600):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

2215 2215 2215


In [ ]:
# Create dataframe and save to csv file
dict_foody6 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df6 = pd.DataFrame(dict_foody6)
df6.head()

,name,review_text,review_score
0,Quán Cơm 2 Món - Âu Cơ,Mở hộp cơm ra thấy con sâu còn nghoe nguẩy. Ch...,1.0
1,Quán Cơm 2 Món - Âu Cơ,"Mình mua tại quán, quán bự mà món ăn nhìn khôn...",4.2
2,Quán Cơm 2 Món - Âu Cơ,"- Cơm ngon , món đậm đà , thường cho thêm rau ...",9.2
3,Quán Cơm 2 Món - Âu Cơ,Mình đã sợ nên ghi chú rất rỏ ràng yêu cầu đổi...,2.6
4,Quán Cơm 2 Món - Âu Cơ,"Quán tuy nhỏ nhưng chất lương ok , mình ăn từ ...",10


In [ ]:
df6.shape

In [ ]:
df6.to_csv('data/foodyData6.csv')

# Foody_Hanoi

# 1. Get links and names of restaurant

In [ ]:
driver.get('https://www.foody.vn/ha-noi')

In [ ]:
# lấy tên và link của các nhà hàng
for i in range (1,50):
    try:
        driver.find_elements_by_class_name('fd-btn-more')[0].click()
        sleep(2)
    except IndexError:
        pass
    continue
res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")
res_names = [x.text for x in res]
links = [x.get_attribute('href') for x in res]

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2824142087.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2824142087.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")


In [ ]:
print('Số lượng nhà hàng',len(res_names))
res_names[:5]

Số lượng nhà hàng 600


['Lemon House - Hà Đông',
 'Pilot Restaurant - Lạc Long Quân',
 'El Gaucho Steakhouse - Tràng Tiền',
 'Xuân Chu Vườn - Lệ Mật',
 'Quán Huế - Bún Bò Huế - Nguyễn Trãi']

In [ ]:
# add thêm '/binh-luan' ở cuối link để truy cập vào phần bình luận của mỗi nhà hàng
links_bl = []
for l in links:
    links_bl.append(l +'/binh-luan')
print('Số lượng link:',len(links_bl))
links_bl[:5]

Số lượng link: 600


['https://www.foody.vn/ha-noi/lemon-house-ha-dong/binh-luan',
 'https://www.foody.vn/ha-noi/pilot-restaurant-lac-long-quan/binh-luan',
 'https://www.foody.vn/ha-noi/el-gaucho-steakhouse-trang-tien/binh-luan',
 'https://www.foody.vn/ha-noi/xuan-chu-vuon-le-mat/binh-luan',
 'https://www.foody.vn/ha-noi/quan-hue-bun-bo-hue-nguyen-trai.yaz7bm/binh-luan']

In [ ]:
# Save results to csv
with open('data/foody_links_bl2.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(links_bl)

with open('data/foody_res_names2.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(res_names)

# 2. Review & Score craping

In [ ]:
# load links
links_bl_load = pd.read_csv('data/foody_links_bl2.csv', header=None)
links_bl_load = list(links_bl_load.iloc[0,:].values)
links_bl_load[:5]

['https://www.foody.vn/ha-noi/lemon-house-ha-dong/binh-luan',
 'https://www.foody.vn/ha-noi/pilot-restaurant-lac-long-quan/binh-luan',
 'https://www.foody.vn/ha-noi/el-gaucho-steakhouse-trang-tien/binh-luan',
 'https://www.foody.vn/ha-noi/xuan-chu-vuon-le-mat/binh-luan',
 'https://www.foody.vn/ha-noi/quan-hue-bun-bo-hue-nguyen-trai.yaz7bm/binh-luan']

In [ ]:
# load restaurant names
res_names_load = pd.read_csv('data/foody_res_names.csv', header=None)
res_names_load = list(res_names_load.iloc[0,:].values)
res_names_load[:5]

In [ ]:
len(links_bl_load), len(res_names_load)

(600, 600)

## Data scraping 7

In [ ]:
# data scraping 100 nhà hàng đầu tiên [0:100]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(0,100):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/1078799695.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/1078799695.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/1078799695.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1745 1744 1744


In [ ]:
# dictionary_products
dictionary_test1 = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test1 = pd.DataFrame(dictionary_test1)

In [ ]:
df_test1['len_review'] = df_test1.review_text.apply(lambda x: len(x))
df_test1['len_score'] = df_test1.review_score.apply(lambda x: len(x))

In [ ]:
df_test1.head()

,name,review_text,review_score,len_review,len_score
0,[Chícc Chícc - Gà Rán Hàn Quốc],[Các món đều khá ngon ở mức giá 20-25k. Bánh t...,[5.0],1,1
1,[Thành Mập - Chân Gà Rút Xương Ngâm Sả Tắc - B...,[Đang thấy quán được review sống ảo nhiều nên ...,"[6.4, 9.0]",2,2
2,"[TocoToco Bubble Tea - Phạm Hùng, TocoToco Bub...",[2 cảm giác đầu tiên khi vào quán là anh béo q...,"[8.2, 8.2, 8.0, 6.6, 6.4, 7.2, 7.6, 7.6, 8.8, ...",36,36
3,"[2 Béo - Bánh Sầu Riêng - Shop Online, 2 Béo -...",[Ở đây chuyên đặc sản về rắn :)) còn đc xem bi...,"[8.0, 7.2, 7.2]",3,3
4,"[Bánh 9 Sạch - Bánh Sầu Riêng - Shop Online, B...",[Nay mình lên Now để đặt bún bò và có đặt thêm...,"[9.6, 10, 9.8, 10, 10]",5,5


In [ ]:
# tìm nhà hàng có len_review và len_score không bằng nhau
x = df_test1[df_test1['len_review']!=df_test1['len_score'] ]
x

,name,review_text,review_score,len_review,len_score
72,"[Ole Restaurant Sài Gòn, Ole Restaurant Sài Gò...","[Thèm thịt là ghé ngay Gogi, đó là 1 thói quen...","[9.8, 8.4, 9.8, 10, 2.4, 10, 10, 10, 10, 10, 9...",17,16


drop nhà hàng này

In [ ]:
del reviews_all[72]

In [ ]:
del scores_all[72]

In [ ]:
del res_names_all[72]

In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1728 1728 1728


In [ ]:
# Create dataframe and save to csv file
dict_foody7 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df7 = pd.DataFrame(dict_foody7)
df7.head()

,name,review_text,review_score
0,Chícc Chícc - Gà Rán Hàn Quốc,Các món đều khá ngon ở mức giá 20-25k. Bánh tr...,5.0
1,Thành Mập - Chân Gà Rút Xương Ngâm Sả Tắc - Bạ...,Đang thấy quán được review sống ảo nhiều nên m...,6.4
2,Thành Mập - Chân Gà Rút Xương Ngâm Sả Tắc - Bạ...,Ngay gần nhà nên là hnay rủ mấy chị đẹp nhà mì...,9.0
3,TocoToco Bubble Tea - Phạm Hùng,2 cảm giác đầu tiên khi vào quán là anh béo qu...,8.2
4,TocoToco Bubble Tea - Phạm Hùng,"El Gaucho nằm ở vị trí khá đắc địa, trên con đ...",8.2


In [ ]:
df7.shape

(1728, 3)

In [ ]:
df7.to_csv('data/foodyData7.csv')

## Data scraping 8

In [ ]:
# data scraping 100 nhà hàng tiếp theo [100:200]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(100,200):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/3014958266.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1729 1728 1728


In [ ]:
# dictionary_products
dictionary_test = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test = pd.DataFrame(dictionary_test)
df_test['len_review'] = df_test.review_text.apply(lambda x: len(x))
df_test['len_score'] = df_test.review_score.apply(lambda x: len(x))

In [ ]:
# tìm nhà hàng có len_review và len_score không bằng nhau
df_test[df_test['len_review']!=df_test['len_score'] ]

,name,review_text,review_score,len_review,len_score
77,"[Teng Teng - Taiwan Tea - Cây Keo, Teng Teng -...",[Lâu không đi ăn bánh cuốn nóng Hà Nội phố.hô...,"[10, 7.6, 6.4, 6.0, 5.6, 1.0, 4.6, 10, 1.0, 5....",44,43


In [ ]:
# reviews_all[77]
# scores_all[77]
# res_names_all[77]

In [ ]:
del reviews_all[77]
del scores_all[77]
del res_names_all[77]

In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1685 1685 1685


In [ ]:
# Create dataframe and save to csv file
dict_foody8 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df8 = pd.DataFrame(dict_foody8)
df8.head()

,name,review_text,review_score
0,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Thật sự đã đi ngủ rồi nhưng cảm thấy đói bực q...,1.0
1,Trà Sữa Feeling Tea - Đường 3 Tháng 2,"Chất lượng đồ ăn thì cảm thấy tạm, không có gì...",5.8
2,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Mình rất thích đồ nhật và đã ăn ở nhiều nhà hà...,4.0
3,Trà Sữa Feeling Tea - Đường 3 Tháng 2,Hồi trước đó ăn Sushi Kei thì toàn ăn những th...,8.2
4,Trà Sữa Feeling Tea - Đường 3 Tháng 2,"menu rất phong phú, đa dạng, giá cả lại hợp lí...",8.6


In [ ]:
df8.shape

(1685, 3)

In [ ]:
df8.to_csv('data/foodyData8.csv')

## Data scraping 9

In [ ]:
# data scraping 100 nhà hàng tiếp theo [200:300]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(200,300):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/910036964.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1800 1800 1800


In [ ]:
# Create dataframe and save to csv file
dict_foody9 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df9 = pd.DataFrame(dict_foody9)
df9.head()

,name,review_text,review_score
0,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,"Nhà mình hay đãi tiệc ở quán này, đồ ăn ngon l...",9.6
1,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Thấy có những món lạ lạ ở quán mà chưa được th...,9.4
2,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,"Vài chai beer cùng vài món mồi ngon, ngồi ở Vi...",9.6
3,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Thích thú với những món ngon như lòng cá vitam...,9.2
4,Hủ Tiếu Chay Cây Đề - Diệp Minh Châu,Tìm được quán nhậu giá cả hợp lý mà không gian...,9.4


In [ ]:
df9.shape

(1800, 3)

In [ ]:
df9.to_csv('data/foodyData9.csv')

## Data scraping 10

In [ ]:
# data scraping 100 nhà hàng tiếp theo [300:400]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(300,400):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2579985156.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1724 1725 1724


In [ ]:
# dictionary_products
dictionary_test = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test = pd.DataFrame(dictionary_test)
df_test['len_review'] = df_test.review_text.apply(lambda x: len(x))
df_test['len_score'] = df_test.review_score.apply(lambda x: len(x))

In [ ]:
# tìm nhà hàng có len_review và len_score không bằng nhau
df_test[df_test['len_review']!=df_test['len_score'] ]

,name,review_text,review_score,len_review,len_score
64,"[SuSu Quán - Món Ăn Hàn Quốc, SuSu Quán - Món ...","[Trân châu ko bị cứng, mặc dù mua về mình chưa...","[9.4, 7.4, 9.6, 7.8, 9.8, 9.6, 7.2, 7.0, 10, 9...",32,33


In [ ]:
del reviews_all[64]
del scores_all[64]
del res_names_all[64]

In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1692 1692 1692


In [ ]:
# Create dataframe and save to csv file
dict_foody10 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df10 = pd.DataFrame(dict_foody10)
df10.head()

,name,review_text,review_score
0,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Tối Hôm 15/2 mình có đi ngang qua quán phố mì ...,8.0
1,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Quán mỳ cay gần trường cấp 3 của mình nên hôm ...,6.4
2,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Được bạn mời mọc xuống Nhổn chơi và sau 1 hồi ...,5.6
3,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,"- Vị trí: Ngay mặt đường, dễ tìm. Nhưng ko hoà...",6.8
4,Bếp Rùa - Chân Gà Rút Xương Ngâm Sả Tắc - Shop...,Địa điểm: Nằm mặt đường ngay chỗ Gold Mark Cit...,8.0


In [ ]:
df10.shape

(1692, 3)

In [ ]:
df10.to_csv('data/foodyData10.csv')

## Data scraping 11

In [ ]:
# data scraping 100 nhà hàng tiếp theo [400:500]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(400,500):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/2010455740.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1900 1900 1900


In [ ]:
# Create dataframe and save to csv file
dict_foody11 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df11 = pd.DataFrame(dict_foody11)
df11.head()

,name,review_text,review_score
0,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,Đồ ăn thì cũng ở mức trung bình\nTuy nhiên riê...,1.2
1,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,Vừa mới ăn hồi chiều xong mà giờ vẫn ngang nga...,7.6
2,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,Giá tiền sinh viên nên cũng không đòi hỏi chất...,7.4
3,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,Nhiều món. Ăn tạm được ngon nhất là khoai vs n...,7.2
4,EMi Shop - Bánh Bao Nguyên Cám Healthy & Bánh ...,"Tất cả đều như đồ cũ , ỉu",4.4


In [ ]:
df11.shape

(1900, 3)

In [ ]:
df11.to_csv('data/foodyData11.csv')

## Data scraping 12

In [ ]:
# data scraping 100 nhà hàng tiếp theo [500:600]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(500,600):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_4170/4197858552.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1672 1672 1672


In [ ]:
# Create dataframe and save to csv file
dict_foody12 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df12 = pd.DataFrame(dict_foody12)
df12.head()

,name,review_text,review_score
0,Quán Cơm 2 Món - Âu Cơ,"Mình cũng hay đi ăn quán thịt dê Tuấn Hằng, tu...",6.8
1,Quán Cơm 2 Món - Âu Cơ,Có được món nầm là khá chút. Rau xào thì không...,2.4
2,Quán Cơm 2 Món - Âu Cơ,Món ăn của quán mà mình ăn qua đều không đáp ứ...,2.0
3,Quán Cơm 2 Món - Âu Cơ,Mình đi ăn ở đây trên 3 lần rồi. Đồ ăn rất ngo...,8.4
4,Quán Cơm 2 Món - Âu Cơ,"Quán này có món dê tái rát ngon, dê ngọt và mề...",7.0


In [ ]:
df12.shape

(1672, 3)

In [ ]:
df12.to_csv('data/foodyData12.csv')

In [ ]:
# remove = "Đây là chi nhánh gần trung tâm nhất, mới nhất và cảm giác cũng hịn nhất. Sau 1 thời gian Tràng Tiền Plaza nói không với các tiệm ăn uống thì phải nói nơi đây dần biến thành chốn sang chảnh chỉ dành cho dân nhiều tiền mua đồ hiệu hoặc là khách du lịch vào thăm quan vì như mình ở HN mà bảo vào TT thì mặt kiểu 'ủa có gì mà vào'. Vài tháng trở lại đây thấy xuất hiện các quán ăn nhẹ, rồi CGV đã thấy mô hình đang dần thân thiện hơn, dự là sẽ dần hot trở lại như xưa.\nKhông gian của khá rộng rãi, 1 bên kia là quán đồ hải sản, cùng tầng là rạp phim với Donut, nguyên tầng dành cho tín đồ ăn uống. Cách trang trí bày biện cảm giác rất Hàn, có sự chỉn chu đáng kể. Đi cuối tuần nhớ gọi đặt bàn vì đông đáo để luôn, mình đi nhóm lớn sẽ được xếp phòng bên trong cũng đỡ bị ồn ào phần nào.\nLâu không Gogi thấy set nướng có món mời trông hấp dẫn dã man, miếng thịt tươi đẹp. Người ta hay bảo ăn nướng thì nên chọn thịt tươi không có gia vị sẽ biết được thịt ngon hay không cũng đúng. Cơ mà sườn tẩm mật ong ở đây là cực phẩm không thể không thử đó, chả cần chấm thêm gì em nó đã đủ ngon.\nNhân viên phục vụ khá ổn, không phải đợi quá lâu. Duy có điều là phải mất công gửi xe, đi lên tầng mà lại mất phí gửi nữa."

In [ ]:
# del reviews_all2[reviews_all2.index(remove)]
# len(reviews_all2)

# Foody_Danang

# 1. Get links and names of restaurant

In [ ]:
driver.get('https://www.foody.vn')

In [ ]:
# lấy tên và link của các nhà hàng
for i in range (1,50):
    try:
        driver.find_elements_by_class_name('fd-btn-more')[0].click()
        sleep(2)
    except IndexError:
        pass
    continue
res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")
res_names = [x.text for x in res]
links = [x.get_attribute('href') for x in res]

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/2824142087.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/2824142087.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")


In [ ]:
print('Số lượng nhà hàng',len(res_names))
res_names[:5]

Số lượng nhà hàng 624


['Bánh Sầu Riêng 9 Sạch - Lê Lợi',
 'Bún Bà Thông - Hoàng Văn Thụ',
 'Hải Sản Bé Ni 2',
 'Highlands Coffee - Điện Biên Phủ',
 'Như Ý Quán - Bún Thịt Nướng & Nem Lụi']

In [ ]:
# add thêm '/binh-luan' ở cuối link để truy cập vào phần bình luận của mỗi nhà hàng
links_bl = []
for l in links:
    links_bl.append(l +'/binh-luan')
print('Số lượng link:',len(links_bl))
links_bl[:5]

Số lượng link: 624


['https://www.foody.vn/da-nang/banh-sau-rieng-9-sach-le-loi/binh-luan',
 'https://www.foody.vn/da-nang/bun-99-hoang-van-thu/binh-luan',
 'https://www.foody.vn/da-nang/hai-san-be-ni/binh-luan',
 'https://www.foody.vn/da-nang/highlands-coffee-dien-bien-phu/binh-luan',
 'https://www.foody.vn/da-nang/nhu-y-quan-bun-thit-nuong-nem-lui/binh-luan']

In [ ]:
# Save results to csv
with open('data/foody_links_bl3.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(links_bl)

with open('data/foody_res_names3.csv', 'wt') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(res_names)

# 2. Review & Score scraping

In [ ]:
# load links
links_bl_load = pd.read_csv('data/foody_links_bl3.csv', header=None)
links_bl_load = list(links_bl_load.iloc[0,:].values)
links_bl_load[:5]

['https://www.foody.vn/da-nang/banh-sau-rieng-9-sach-le-loi/binh-luan',
 'https://www.foody.vn/da-nang/bun-99-hoang-van-thu/binh-luan',
 'https://www.foody.vn/da-nang/hai-san-be-ni/binh-luan',
 'https://www.foody.vn/da-nang/highlands-coffee-dien-bien-phu/binh-luan',
 'https://www.foody.vn/da-nang/nhu-y-quan-bun-thit-nuong-nem-lui/binh-luan']

In [ ]:
# load restaurant names
res_names_load = pd.read_csv('data/foody_res_names3.csv', header=None)
res_names_load = list(res_names_load.iloc[0,:].values)
res_names_load[:5]

['Bánh Sầu Riêng 9 Sạch - Lê Lợi',
 'Bún Bà Thông - Hoàng Văn Thụ',
 'Hải Sản Bé Ni 2',
 'Highlands Coffee - Điện Biên Phủ',
 'Như Ý Quán - Bún Thịt Nướng & Nem Lụi']

In [ ]:
len(links_bl_load), len(res_names_load)

(624, 624)

## Data scraping 13

In [ ]:
# data scraping 100 nhà hàng đầu tiên [0:100]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(0,100):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/1078799695.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/1078799695.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/1078799695.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

2157 2157 2157


In [ ]:
# Create dataframe and save to csv file
dict_foody13 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df13 = pd.DataFrame(dict_foody13)

In [ ]:
df13.head()

,name,review_text,review_score
0,Bánh Sầu Riêng 9 Sạch - Lê Lợi,"Bánh ngon, mềm và thơm nứt mũi luôn ạ ☺️☺️☺️",10
1,Bánh Sầu Riêng 9 Sạch - Lê Lợi,"Bánh ngon, mềm và thơm nứt mũi luôn ạ ☺️☺️☺️",10
2,Bánh Sầu Riêng 9 Sạch - Lê Lợi,"Bánh ngon, mềm và thơm nứt mũi luôn ạ ☺️☺️☺️",10
3,Bánh Sầu Riêng 9 Sạch - Lê Lợi,"Bánh ngon, mềm và thơm nứt mũi luôn ạ ☺️☺️☺️",10
4,Bánh Sầu Riêng 9 Sạch - Lê Lợi,"Bánh ngon, mềm và thơm nứt mũi luôn ạ ☺️☺️☺️",10


In [ ]:
df13.shape

(2157, 3)

In [ ]:
df13.to_csv('data/foodyData13.csv')

## Data scraping 14

In [ ]:
# data scraping 100 nhà hàng đầu tiên [100:200]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(100,200):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/784232489.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/784232489.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/784232489.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1877 1877 1877


In [ ]:
# Create dataframe and save to csv file
dict_foody14 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df14 = pd.DataFrame(dict_foody14)

In [ ]:
df14.head()

,name,review_text,review_score
0,Bà Bê - Bánh Bèo & Bánh Ướt,Mỗi lần đi chợ Cồn m đều gé ăn\nChiều quán bán...,10
1,Bà Bê - Bánh Bèo & Bánh Ướt,"Đây là quán bánh bèo lâu năm gần nhà mình, hồi...",7.0
2,Bà Bê - Bánh Bèo & Bánh Ướt,Quán này bán lâu lắm rồi..chỉ nhớ là tình cờ b...,8.6
3,Bin - Nước Dừa & Bánh Tráng Dừa,"Hình như đường nguyễn văn linh thay số , cả cũ...",8.2
4,Bin - Nước Dừa & Bánh Tráng Dừa,Trước hay ăn quán ni ở ngoài đường Nguyễn Văn ...,8.4


In [ ]:
df14.to_csv('data/foodyData14.csv')

## Data scraping 15

In [ ]:
# data scraping 100 nhà hàng đầu tiên [200:300]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(200,300):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3884381087.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3884381087.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3884381087.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1553 1553 1553


In [ ]:
# Create dataframe and save to csv file
dict_foody15 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df15 = pd.DataFrame(dict_foody15)

In [ ]:
df15.head()

,name,review_text,review_score
0,Ahhh Sầu Riêng - Đống Đa,Không hiểu sao trời nóng là mình lại thích uốn...,8.0
1,Nhà Rốt - Nước Mía Sầu Riêng,Món này rất dễ gây nghiện. Ngon lắm mọi người....,9.8
2,Nhà Rốt - Nước Mía Sầu Riêng,Nước mía gần cty mình nên mình cũng hay đặt uố...,4.0
3,Nhà Rốt - Nước Mía Sầu Riêng,"Ấn tượng đầu tiên với mình là cái tên, vừa lạ ...",9.0
4,Nhà Rốt - Nước Mía Sầu Riêng,"Nghe tên món thấy lạ lạ nên đi uống thử, nước ...",6.2


In [ ]:
df15.to_csv('data/foodyData15.csv')

## Data scraping 16

In [ ]:
# data scraping 100 nhà hàng đầu tiên [300:400]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(300,400):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/32996707.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/32996707.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/32996707.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1746 1746 1746


In [ ]:
# Create dataframe and save to csv file
dict_foody16 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df16 = pd.DataFrame(dict_foody16)

In [ ]:
df16.head()

,name,review_text,review_score
0,Takuhai Sushi,"Phải nói sao nhỉ. Trước khi ăn chỗ này, mình c...",9.0
1,Takuhai Sushi,"Tiệm sushsi bí ẩn chỉ dành cho ""đại gia"".\nMìn...",7.0
2,Takuhai Sushi,Món ăn ở đây quá tuyệt vời.\nđồ ăn dc chính đầ...,8.0
3,Takuhai Sushi,Quán có không gian khá nhỏ nhưng lại có vẻ ấm ...,8.0
4,Takuhai Sushi,Ngoài việc đồ ăn tươi ngon chế biến đơn giản n...,6.8


In [ ]:
df16.to_csv('data/foodyData16.csv')

## Data scraping 17

In [ ]:
# data scraping 100 nhà hàng đầu tiên [400:500]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(400,500):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/194666832.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/194666832.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/194666832.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

2011 2010 2010


In [ ]:
# dictionary_products
dictionary_test1 = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test1 = pd.DataFrame(dictionary_test1)

In [ ]:
df_test1['len_review'] = df_test1.review_text.apply(lambda x: len(x))
df_test1['len_score'] = df_test1.review_score.apply(lambda x: len(x))

In [ ]:
df_test1[df_test1['len_review'] != df_test1['len_score']]

,name,review_text,review_score,len_review,len_score
59,"[Cơm Chay Đà Nẵng, Cơm Chay Đà Nẵng, Cơm Chay ...","[Mình đã ghé qua nhiều lần, quán bán nhiều món...","[9.8, 2.0, 7.0, 5.0, 2.4, 8.0, 4.6, 9.2, 1.0, ...",60,59


In [ ]:
del reviews_all[59]
del scores_all[59]
del res_names_all[59]

In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1951 1951 1951


In [ ]:
dict_foody17 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df17 = pd.DataFrame(dict_foody17)

In [ ]:
df17.head()

,name,review_text,review_score
0,Quán Nhân - Cháo Vịt,"Thịt vịt khô, thơm, ngọt ngon. Cháo cũng vừa ă...",8.4
1,Bánh Tráng Nướng Đà Lạt NTP,Bánh tráng nướng bỏ nhiều mayonaise nên hơi ch...,7.8
2,Bánh Tráng Nướng Đà Lạt NTP,Quán nằm ngay đầu hẻm 78 Ph. Thanh. Chỗ ngồi l...,6.4
3,Bánh Tráng Nướng - Phan Châu Trinh,Chiều chiều buồn miệng thì ăn gì nhỉ. 😜 🔆 Kiếm...,5.0
4,Bánh Tráng Nướng - Phan Châu Trinh,Mình ăn cũng gần hết các quán bánh tráng có ti...,3.8


In [ ]:
df17.to_csv('data/foodyData17.csv')

## Data scraping 18

In [ ]:
# data scraping 124 nhà hàng cuối cùng [500:624]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(500,624):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3545209870.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3545209870.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_15460/3545209870.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [ ]:
# flat the lists and check their length
res_names_all2 = sum(res_names_all, [])
reviews_all2 = sum(reviews_all, [])
scores_all2 = sum(scores_all, [])
print(len(reviews_all2), len(scores_all2), len(res_names_all2))

1993 1993 1993


In [ ]:
dict_foody18 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df18 = pd.DataFrame(dict_foody18)

In [ ]:
df18.head()

,name,review_text,review_score
0,Y Vân Quán - Bún - Mì Quảng & Bánh Xèo,"Tuy bún đầy đủ rau, thịt nhưng nước nấu quá dở...",4.4
1,Xôi Hong 7 Màu,Xa so với trung tâm nên dù có thích cũng lâu l...,7.0
2,Xôi Hong 7 Màu,"Xôi ngon nhá, mặn có, ngọt có, quán chỉ bán bu...",7.2
3,Xôi Hong 7 Màu,Mình ở ks gần nên sáng quyết định đi bộ sang m...,7.4
4,Xôi Hong 7 Màu,Quán này chỉ bán buổi sáng thôi chứ không bán ...,9.0


In [ ]:
df18.to_csv('data/foodyData18.csv')

# Foody_HaiPhong

In [39]:
# get links and names of restaurant
def get_links_names(driver):
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()
            sleep(2)
        except IndexError:
            pass
        continue
    res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")
    res_names = [x.text for x in res]
    links = [x.get_attribute('href') for x in res]
    print('Số lượng nhà hàng',len(res_names))
    print('Số lượng link:',len(links))

#     # add thêm '/binh-luan' ở cuối link để truy cập vào phần bình luận của mỗi nhà hàng
#     links_bl = []
#     for l in links:
#         links_bl.append(l +'/binh-luan')
#     print('Số lượng link:',len(links_bl))
    return links, res_names

In [5]:
# Save results to csv
def save_links_names(data, path):
    with open(path, 'wt') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(data)

In [80]:
# data scraping 100 nhà hàng đầu tiên [0:100]
def data_scraping(res_range, links_bl_load, res_names_load):
    reviews_all = []
    scores_all = []
    res_names_all = []
    for e in res_range:
        driver.get(links_bl_load[e])
        for i in range (1,50):
            try:
                driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
                sleep(2)
            except IndexError:
                pass
            continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)
    return res_names_all, reviews_all, scores_all

In [7]:
# flat the lists and check their length
def flaten_list(res_names_all, reviews_all, scores_all):
    res_names_all2 = sum(res_names_all, [])
    reviews_all2 = sum(reviews_all, [])
    scores_all2 = sum(scores_all, [])
    print(len(reviews_all2), len(scores_all2), len(res_names_all2))
    return res_names_all2, reviews_all2, scores_all2

In [44]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/elliedam/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [45]:
driver.get('https://www.foody.vn/hai-phong')

In [46]:
# get links and names of restaurant
links, res_names = get_links_names(driver)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1303928511.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1303928511.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res = driver.find_elements_by_xpath("//div[contains(@class,'fd-text-ellip')]/a")


Số lượng nhà hàng 600
Số lượng link: 600


In [58]:
del links[500] #3, 127

In [62]:
del res_names[3]
del res_names[127]
del res_names[500]

In [63]:
len(links), len(res_names)

(597, 597)

In [64]:
# add thêm '/binh-luan' ở cuối link để truy cập vào phần bình luận của mỗi nhà hàng
links_bl = []
for l in links:
    links_bl.append(l +'/binh-luan')
print('Số lượng link:',len(links_bl))

Số lượng link: 597


In [65]:
# Save results to csv
save_links_names(links_bl, 'data/foody_links_bl4.csv')
save_links_names(res_names, 'data/foody_res_names4.csv')

In [75]:
# Load files
links_bl_load = pd.read_csv('data/foody_links_bl4.csv', header=None)
res_names_load = pd.read_csv('data/foody_res_names4.csv', header=None)

In [76]:
links_bl_load = list(links_bl_load.iloc[0,:].values)
res_names_load = list(res_names_load.iloc[0,:].values)

In [84]:
# data scraping 100 nhà hàng đầu tiên [0:100]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(0,100):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1078799695.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1078799695.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/1078799695.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [85]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1901 1901 1901


In [86]:
dict_foody19 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df19 = pd.DataFrame(dict_foody19)
print(df19.shape)
df19.head()

(1901, 3)


,name,review_text,review_score
0,Ốc Các Loại - Thiên Lôi,"Lâu lâu không ăn ốc lại lên cơn thèm, thế là r...",7.0
1,Ốc Các Loại - Thiên Lôi,Ốc hẳn là món ăn vặt khoải khẩu của đa số mọi ...,7.0
2,Ốc Các Loại - Thiên Lôi,"Ngon ngon ngon !!!\nMình khá thích ăn ốc, được...",8.2
3,Ốc Các Loại - Thiên Lôi,"Quán này nằm ngay mặt đường Thiên Lôi, ngay đo...",7.8
4,Ốc Các Loại - Thiên Lôi,"Quán hơi xa nhà mình xíu, nằm ở mặt đường Thiê...",8.0


In [87]:
df19.to_csv('data/foodyData19.csv')

In [88]:
# data scraping 100 nhà hàng đầu tiên [100:200]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(100,200):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/784232489.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/784232489.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/784232489.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [89]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1847 1844 1844


In [104]:
# dictionary_products
dictionary_test1 = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test1 = pd.DataFrame(dictionary_test1)

In [105]:
df_test1['len_review'] = df_test1.review_text.apply(lambda x: len(x))
df_test1['len_score'] = df_test1.review_score.apply(lambda x: len(x))

In [106]:
df_test1[df_test1['len_review'] != df_test1['len_score']]

,name,review_text,review_score,len_review,len_score
13,[Quán Ăn Vặt - Cát Cụt],[Cháo trai 10k/bát kèm cả quẩy nữa 😉 #themlanh...,[6.6],2,1


In [107]:
del res_names_all[13]
del scores_all[13]
del reviews_all[13]

In [108]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1628 1628 1628


In [109]:
dict_foody20 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df20 = pd.DataFrame(dict_foody20)
print(df20.shape)
df20.head()

(1628, 3)


,name,review_text,review_score
0,Lotteria - Hải Phòng Trần Phú,Giao toàn thiếu đồ mà ko báo đêm rồi nhận hàng...,1.0
1,Lotteria - Hải Phòng Trần Phú,Có ai nghiện gà sốt HS ở đây giống mình ko huh...,7.4
2,Lotteria - Hải Phòng Trần Phú,Thái độ phục vụ của các chị nhân viên vô cùng ...,1.0
3,Lotteria - Hải Phòng Trần Phú,Thu ngân quá chán\nĐứng chờ mãi k thấy chị thu...,4.2
4,Lotteria - Hải Phòng Trần Phú,Rất lâu mới quay lại Lotteria*** Phải nói mình...,3.4


In [110]:
df20.to_csv('data/foodyData20.csv')

In [111]:
# data scraping 100 nhà hàng đầu tiên [200:300]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(200,300):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3884381087.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3884381087.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3884381087.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [112]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1224 1224 1224


In [113]:
dict_foody21 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df21 = pd.DataFrame(dict_foody21)
print(df21.shape)
df21.head()

(1224, 3)


,name,review_text,review_score
0,Bulgogi - Đồ Nướng Than Hoa - Văn Cao,#review #homnayviangi \n#bulgogivancao #đầungõ...,8.8
1,Bulgogi - Đồ Nướng Than Hoa - Văn Cao,Mọi lần mình hay đến buổi tối nên đc ngồi trên...,5.0
2,Bulgogi - Đồ Nướng Than Hoa - Văn Cao,"Mình đến quán tầm trưa, quán khá dễ tìm, hôm đ...",7.4
3,Bulgogi - Đồ Nướng Than Hoa - Văn Cao,Được ngày rủ cb đi ăn lẩu tưởng lành lạnh tý ă...,7.6
4,Bulgogi - Đồ Nướng Than Hoa - Văn Cao,Đợt này bulgogi có khuyến mại bufet lẩu bò 3 n...,8.2


In [114]:
df21.to_csv('data/foodyData21.csv')

In [115]:
# data scraping 100 nhà hàng đầu tiên [300:400]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(300,400):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/32996707.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/32996707.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/32996707.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [116]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1357 1355 1355


In [117]:
# dictionary_products
dictionary_test1 = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test1 = pd.DataFrame(dictionary_test1)
df_test1['len_review'] = df_test1.review_text.apply(lambda x: len(x))
df_test1['len_score'] = df_test1.review_score.apply(lambda x: len(x))
df_test1[df_test1['len_review'] != df_test1['len_score']]

,name,review_text,review_score,len_review,len_score
55,"[Bánh Đúc Tàu - Chợ Máy Đá, Bánh Đúc Tàu - Chợ...",[Được biết ở Hải Phòng có 3 địa chỉ bán bánh đ...,"[7.8, 8.0]",3,2
77,"[Huệ Béo - Bánh Gối & Giá Bể Xào, Huệ Béo - Bá...",[Chợ Cột Đèn ngoài hàng cháo sườn nổi tiếng ra...,"[7.0, 6.4, 8.4, 8.4, 7.6, 8.2, 7.2, 7.4, 8.0, ...",11,10


In [118]:
del res_names_all[55]
del res_names_all[76]
del reviews_all[55]
del reviews_all[76]
del scores_all[55]
del scores_all[76]

In [119]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1343 1343 1343


In [120]:
dict_foody22 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df22 = pd.DataFrame(dict_foody22)
print(df22.shape)
df22.head()

(1343, 3)


,name,review_text,review_score
0,Cô Dung - Trứng Vịt Lộn & Bánh Mì Giò Online,ko ngờ có món này trên now luôn. \nhôm đó thèm...,6.0
1,Cô Dung - Trứng Vịt Lộn & Bánh Mì Giò Online,Ngày mưa gió thấy món trứng vịt lộn ở trên now...,6.0
2,Linh - Bánh Cuốn Cát Dài,"Cuối tuần rảnh được đưa về Hải phòng chơi, và ...",6.2
3,Linh - Bánh Cuốn Cát Dài,"Quán này bán cũng được hơn 20 năm rồi, bánh ở ...",7.8
4,Linh - Bánh Cuốn Cát Dài,"Bánh cuốn ở đây cũng có thâm niên lắm luôn, ph...",8.8


In [121]:
df22.to_csv('data/foodyData22.csv')

In [125]:
# data scraping 100 nhà hàng đầu tiên [400:500]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(400,500):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/194666832.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/194666832.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/194666832.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [126]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1005 1005 1005


In [127]:
dict_foody23 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df23 = pd.DataFrame(dict_foody23)
print(df23.shape)
df23.head()

(1005, 3)


,name,review_text,review_score
0,Bình Cafe,Lượn lờ sau khu big c thấy quá trời quán ăn uố...,8.6
1,Lão Ngư - Hải Sản Các Loại - Nguyễn Đức Cảnh,"Vị ngọt cúa sò, vị béo của phô mai kết hợp với...",10
2,Lão Ngư - Hải Sản Các Loại - Nguyễn Đức Cảnh,Không gian ấm áp - và hơn nữa nhân viên rất th...,9.8
3,Lão Ngư - Hải Sản Các Loại - Nguyễn Đức Cảnh,"Nhà hàng decor đẹp mắt, sạch sẽ, vị trí dễ tìm...",9.6
4,Lão Ngư - Hải Sản Các Loại - Nguyễn Đức Cảnh,"Hôm nay mình ord lẩu Thái ở đây, rất hài lòng....",9.8


In [128]:
df23.to_csv('data/foodyData23.csv')

In [129]:
# data scraping 100 nhà hàng đầu tiên [500:597]
reviews_all = []
scores_all = []
res_names_all = []
for e in range(500,597):
    driver.get(links_bl_load[e])
    for i in range (1,50):
        try:
            driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more 
            sleep(2)
        except IndexError:
            pass
        continue

    review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
    score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score
    reviews = [x.text for x in review]
    scores = [x.text for x in score]
    res_names_dup = [res_names_load[e]] * min(len(reviews),len(scores[:-1]))
    # append
    reviews_all.append(reviews)
    scores_all.append(scores[:-1])
    res_names_all.append(res_names_dup)

/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3977138420.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('fd-btn-more')[0].click()  #load more
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3977138420.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  review = driver.find_elements_by_xpath("//div[contains(@class,'rd-des')]/span") #get reviews
/var/folders/75/xh20dgfj413gnh1kwq6jslz00000gn/T/ipykernel_20620/3977138420.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  score = driver.find_elements_by_xpath("//div[contains(@class,'review-points')]/span") #get score


In [130]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1517 1516 1516


In [131]:
# dictionary_products
dictionary_test1 = {'name':res_names_all,
                      'review_text':reviews_all,
                      'review_score': scores_all}
df_test1 = pd.DataFrame(dictionary_test1)
df_test1['len_review'] = df_test1.review_text.apply(lambda x: len(x))
df_test1['len_score'] = df_test1.review_score.apply(lambda x: len(x))
df_test1[df_test1['len_review'] != df_test1['len_score']]

,name,review_text,review_score,len_review,len_score
65,"[Huyền - Bánh Mì Cay, Chè Thái & Sữa Đậu Nành,...",[Mình k nhớ có phải số 37 không nữa. Chưa bàn ...,"[2.6, 8.6, 7.0, 10, 2.6, 3.4, 8.6, 4.2, 2.8, 6...",100,99


In [132]:
del res_names_all[65]
del reviews_all[65]
del scores_all[65]

In [133]:
# flat the lists and check their length
res_names_all2, reviews_all2, scores_all2 = flaten_list(res_names_all, reviews_all, scores_all)

1417 1417 1417


In [134]:
dict_foody24 = {'name':res_names_all2,
                'review_text':reviews_all2,
                'review_score': scores_all2}
df24 = pd.DataFrame(dict_foody24)
print(df24.shape)
df24.head()

(1417, 3)


,name,review_text,review_score
0,Huy - Bò Nướng & Lẩu,"Quán nằm ngay mặt đường, cũng dễ tìm; không gi...",6.2
1,Huy - Bò Nướng & Lẩu,Chỗ này thì độ nổi tiếng khỏi phải bàn tới rồi...,6.8
2,Huy - Bò Nướng & Lẩu,Quán khá nổi và cực kỳ đông khách !!!! Ngày th...,6.6
3,Huy - Bò Nướng & Lẩu,Thấy mọi ng khen nhiều hôm nay ăn thử\nKhông g...,5.0
4,Huy - Bò Nướng & Lẩu,"Quá khá đông, lúc nào thấy cũng full bàn . Hôm...",4.8


In [135]:
df24.to_csv('data/foodyData24.csv')

# Concat & save final result

In [2]:
import pandas as pd

In [2]:
# Load các df đã save 
foodyData1 = pd.read_csv('data/foodyData1.csv', index_col=0)
foodyData2 = pd.read_csv('data/foodyData2.csv', index_col=0)
foodyData3 = pd.read_csv('data/foodyData3.csv', index_col=0)
foodyData4 = pd.read_csv('data/foodyData4.csv', index_col=0)
foodyData5 = pd.read_csv('data/foodyData5.csv', index_col=0)
foodyData6 = pd.read_csv('data/foodyData6.csv', index_col=0)
foodyData7 = pd.read_csv('data/foodyData7.csv', index_col=0)
foodyData8 = pd.read_csv('data/foodyData8.csv', index_col=0)
foodyData9 = pd.read_csv('data/foodyData9.csv', index_col=0)
foodyData10 = pd.read_csv('data/foodyData10.csv', index_col=0)
foodyData11 = pd.read_csv('data/foodyData11.csv', index_col=0)
foodyData12 = pd.read_csv('data/foodyData12.csv', index_col=0)
foodyData13 = pd.read_csv('data/foodyData13.csv', index_col=0)
foodyData14 = pd.read_csv('data/foodyData14.csv', index_col=0)
foodyData15 = pd.read_csv('data/foodyData15.csv', index_col=0)
foodyData16 = pd.read_csv('data/foodyData16.csv', index_col=0)
foodyData17 = pd.read_csv('data/foodyData17.csv', index_col=0)
foodyData18 = pd.read_csv('data/foodyData18.csv', index_col=0)
foodyData19 = pd.read_csv('data/foodyData19.csv', index_col=0)
foodyData20 = pd.read_csv('data/foodyData20.csv', index_col=0)
foodyData21 = pd.read_csv('data/foodyData21.csv', index_col=0)
foodyData22 = pd.read_csv('data/foodyData22.csv', index_col=0)
foodyData23 = pd.read_csv('data/foodyData23.csv', index_col=0)
foodyData24 = pd.read_csv('data/foodyData24.csv', index_col=0)
foodyData25 = pd.read_csv('data/foodyData25.csv', index_col=0)
foodyData26 = pd.read_csv('data/foodyData26.csv', index_col=0)
foodyData27 = pd.read_csv('data/foodyData27.csv', index_col=0)
foodyData28 = pd.read_csv('data/foodyData28.csv', index_col=0)
foodyData29 = pd.read_csv('data/foodyData29.csv', index_col=0)
foodyData30 = pd.read_csv('data/foodyData30.csv', index_col=0)
foodyData31 = pd.read_csv('data/foodyData31.csv', index_col=0)
foodyData32 = pd.read_csv('data/foodyData32.csv', index_col=0)
foodyData33 = pd.read_csv('data/foodyData33.csv', index_col=0)
foodyData34 = pd.read_csv('data/foodyData34.csv', index_col=0)
foodyData35 = pd.read_csv('data/foodyData35.csv', index_col=0)
foodyData36 = pd.read_csv('data/foodyData36.csv', index_col=0)
foodyData37 = pd.read_csv('data/foodyData37.csv', index_col=0)
foodyData38 = pd.read_csv('data/foodyData38.csv', index_col=0)
foodyData39 = pd.read_csv('data/foodyData39.csv', index_col=0)
foodyData40 = pd.read_csv('data/foodyData40.csv', index_col=0)
foodyData41 = pd.read_csv('data/foodyData41.csv', index_col=0)
foodyData42 = pd.read_csv('data/foodyData42.csv', index_col=0)

In [3]:
# concat dataframes into one
foodyDataFinal = pd.concat([foodyData1, foodyData2, foodyData3, foodyData4, foodyData5, foodyData6,
                           foodyData7, foodyData8, foodyData9, foodyData10, foodyData11, foodyData12,
                           foodyData13, foodyData14, foodyData15, foodyData16, foodyData17, foodyData18,
                           foodyData19, foodyData20, foodyData21, foodyData22, foodyData23, foodyData24,
                           foodyData25, foodyData26, foodyData27, foodyData28, foodyData29, foodyData30,
                           foodyData31, foodyData32, foodyData33, foodyData34, foodyData35, foodyData36,
                           foodyData37, foodyData38, foodyData39, foodyData40, foodyData41, foodyData42])

In [4]:
# reset index
foodyDataFinal.reset_index(inplace=True, drop=True)

In [5]:
foodyDataFinal.shape

(77662, 3)

In [6]:
foodyDataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77662 entries, 0 to 77661
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          77662 non-null  object 
 1   review_text   77662 non-null  object 
 2   review_score  77662 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.8+ MB


In [7]:
foodyDataFinal.head()

,name,review_text,review_score
0,Chícc Chícc - Gà Rán Hàn Quốc,"Gà tắm mắm, phô mai kéo sợi siêu ngon, giá mềm",10.0
1,Chícc Chícc - Gà Rán Hàn Quốc,"Gà BBQ Hàn, phô mai kéo sợi siêu ngon, giá mềm",10.0
2,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán phô mai kéo sợi sốt BBQ, siu ngon",10.0
3,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán kem bơ tỏi, ngon, lạ, béo, giá ổn áp",10.0
4,Chícc Chícc - Gà Rán Hàn Quốc,"Gà rán kem bơ tỏi, ngon, lạ, béo, giá ổn áp",10.0


In [8]:
foodyDataFinal.tail()

,name,review_text,review_score
77657,Phở Hương,Quán chủ yếu là phở nhưng mình lại thích bún b...,8.6
77658,Phở Hương,Quán phở này bán cũng đã khá lâu năm rồi. Mình...,8.0
77659,Phở Hương,Là dân thích phở nên nghe giới thiệu có quán p...,8.0
77660,Phở Hương,Tính đi ăn Tài Hưng mà hết ời nên ghé qua đây ...,7.8
77661,Phở Hương,Đợt tính lên rv mà thấy lạ chưa ai ăn ở đây ha...,8.0


In [9]:
foodyDataFinal.duplicated().sum()

7023

In [10]:
# save final file
foodyDataFinal.to_csv('data/foodyDataFinal.csv')